<a href="https://colab.research.google.com/github/Saputoa21/Machine-Translation/blob/main/filterMT_AdvancedMT_2025W.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/ymoslem/Adaptive-MT-LLM-Fine-tuning/blob/main/Data-Processing-Adaptive-MT.ipynb
#https://github.com/ymoslem/MT-Preparation/tree/main

In [ ]:
!git clone https://github.com/ymoslem/MT-Preparation.git

fatal: destination path 'MT-Preparation' already exists and is not an empty directory.


In [ ]:
%cd MT-Preparation

/MT-Preparation


In [ ]:
!pip3 install --user -r requirements.txt

In [ ]:
%cd ..

/


In [ ]:
!ls

bin			    kaggle	    NGC-DL-CONTAINER-LICENSE  srv
boot			    lib		    opt			      sys
content			    lib32	    proc		      tmp
cuda-keyring_1.1-1_all.deb  lib64	    python-apt		      tools
datalab			    libx32	    python-apt.tar.xz	      usr
dev			    media	    root		      var
etc			    mnt		    run
home			    MT-Preparation  sbin


In [ ]:
!python3 MT-Preparation/filtering/filter.py train.en-de.en train.en-de.de en de

Traceback (most recent call last):
  File "//MT-Preparation/filtering/filter.py", line 152, in <module>
    prepare(source_file, target_file, source_lang, target_lang, lower=False)
  File "//MT-Preparation/filtering/filter.py", line 21, in prepare
    df_source = pd.read_csv(source_file, names=['Source'], sep="\0", quoting=csv.QUOTE_NONE, skip_blank_lines=False, on_bad_lines="skip")
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py",

In [ ]:
!python3 MT-Preparation/filtering/semantic_filter.py train.en-de.en-filtered.en train.en-de.de-filtered.de en de 10000 0.45

Traceback (most recent call last):
  File "//MT-Preparation/filtering/semantic_filter.py", line 3, in <module>
    from sentence_transformers import SentenceTransformer
  File "/usr/local/lib/python3.12/dist-packages/sentence_transformers/__init__.py", line 15, in <module>
    from sentence_transformers.cross_encoder import (
  File "/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/__init__.py", line 3, in <module>
    from .CrossEncoder import CrossEncoder
  File "/usr/local/lib/python3.12/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py", line 16, in <module>
object address  : 0x7cd1c6228d60
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [ ]:
!wc -l *

wc: model_cache: Is a directory
       0 model_cache
wc: MT-Preparation: Is a directory
       0 MT-Preparation
wc: sample_data: Is a directory
       0 sample_data
   50000 train.en-de.de
   48315 train.en-de.de-filtered.de
   47614 train.en-de.de-filtered.de.semantic.de
   50000 train.en-de.en
   48315 train.en-de.en-filtered.en
   47614 train.en-de.en-filtered.en.semantic.en
  291858 total


# **My corpus text**

I have chosen MultiUN (v1) corpus with the language pair EN-RU and size of 11,654,416 sentences. I splitted it into train (80k), validation (10k) and test (10k) sets.

The created files are:

*   UN-train.en-ru.en
*   UN-train.en-ru.ru
*   UN-test.en-ru.en
*   UN-test.en-ru.ru
*   UN-val.en-ru.en
*   UN-val.en-ru.ru

Link: https://opus.nlpl.eu/MultiUN/en&ru/v1/MultiUN


## Corpus splitting

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
file_ru = "/content/drive/MyDrive/Colab Notebooks/ru.txt.gz"
file_en = "/content/drive/MyDrive/Colab Notebooks/en.txt.gz"

In [96]:
import gzip

def extract_first_n_lines(gz_path, n_lines=100000):
    lines = []
    with gzip.open(gz_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= n_lines:
                break
            lines.append(line)
    print(f"Extracted {len(lines):,} lines from {gz_path}")
    return lines

In [97]:
lines_ru = extract_first_n_lines(file_ru, n_lines=100000)
lines_en = extract_first_n_lines(file_en, n_lines=100000)

Extracted 100,000 lines from /content/drive/MyDrive/Colab Notebooks/ru.txt.gz
Extracted 100,000 lines from /content/drive/MyDrive/Colab Notebooks/en.txt.gz


In [98]:
import random

def split_data(lines, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    n = len(lines)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)

    train = lines[:n_train]
    val = lines[n_train:n_train + n_val]
    test = lines[n_train + n_val:]

    return train, val, test

In [99]:
train_ru, val_ru, test_ru = split_data(lines_ru)
train_en, val_en, test_en = split_data(lines_en)

print(len(train_ru))
print(len(val_ru))
print(len(test_ru))

80000
10000
10000


In [100]:
def save_bilingual_splits(prefix="UN", src_lang="en", tgt_lang="ru",
                          train_src=None, train_tgt=None,
                          val_src=None, val_tgt=None,
                          test_src=None, test_tgt=None):

    def save(name, src_data, tgt_data):
        src_file = f"{prefix}.{src_lang}-{tgt_lang}.{name}.{src_lang}"
        tgt_file = f"{prefix}.{src_lang}-{tgt_lang}.{name}.{tgt_lang}"

        with open(src_file, "w", encoding="utf-8") as f_src:
            f_src.writelines(src_data)
        with open(tgt_file, "w", encoding="utf-8") as f_tgt:
            f_tgt.writelines(tgt_data)

        print(f"Saved {src_file} ({len(src_data):,} lines)")
        print(f"Saved {tgt_file} ({len(tgt_data):,} lines)")

    if train_src and train_tgt:
        save("train", train_src, train_tgt)
    if val_src and val_tgt:
        save("val", val_src, val_tgt)
    if test_src and test_tgt:
        save("test", test_src, test_tgt)


In [101]:
save_bilingual_splits(
    prefix="UN",
    src_lang="en",
    tgt_lang="ru",
    train_src=train_en, train_tgt=train_ru,
    val_src=val_en, val_tgt=val_ru,
    test_src=test_en, test_tgt=test_ru
)

Saved UN.en-ru.train.en (80,000 lines)
Saved UN.en-ru.train.ru (80,000 lines)
Saved UN.en-ru.val.en (10,000 lines)
Saved UN.en-ru.val.ru (10,000 lines)
Saved UN.en-ru.test.en (10,000 lines)
Saved UN.en-ru.test.ru (10,000 lines)


## Corpus preparation

In [102]:
!git clone https://github.com/ymoslem/MT-Preparation.git

fatal: destination path 'MT-Preparation' already exists and is not an empty directory.


In [103]:
%cd MT-Preparation

/MT-Preparation


In [104]:
!pip3 install --user -r requirements.txt

In [105]:
%cd ..

/


In [106]:
!ls

 bin			      tmp
 boot			      tools
 content		      UN.en-ru.test.en
 cuda-keyring_1.1-1_all.deb   UN.en-ru.test.en-filtered.en
 datalab		     'UN.en-ru.test, en.txt'
 dev			      UN.en-ru.test.ru
 etc			      UN.en-ru.test.ru-filtered.ru
 home			     'UN.en-ru..test, ru.txt'
 kaggle			     'UN.en-ru.test, ru.txt'
 lib			      UN.en-ru._test.txt
 lib32			      UN.en-ru.train.en
 lib64			      UN.en-ru.train.en-filtered.en
 libx32			      UN.en-ru.train.en-filtered.en.semantic.en
 media			      UN.en-ru.train.en.txt
 mnt			      UN.en-ru.train.ru
 model_cache		      UN.en-ru.train.ru-filtered.ru
 MT-Preparation		      UN.en-ru.train.ru-filtered.ru.semantic.ru
 NGC-DL-CONTAINER-LICENSE     UN.en-ru..train.ru.txt
 opt			      UN.en-ru.train.ru.txt
 proc			      UN.en-ru._train.txt
 python-apt		      UN.en-ru.val.en
 python-apt.tar.xz	      UN.en-ru.val.en-filtered.en
 root			      UN.en-ru.val.en.txt
 run			      UN.en-ru.val.ru
 sample_corpus_test.txt       UN.en-ru.val.ru-filtered.ru
 

In [107]:
!python3 MT-Preparation/filtering/filter.py UN.en-ru.train.en UN.en-ru.train.ru en ru

Dataframe shape (rows, columns): (80000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 80000
--- Duplicates Deleted			--> Rows: 79972
--- Source-Copied Rows Deleted		--> Rows: 79971
--- Too Long Source/Target Deleted	--> Rows: 39271
--- Too Short Source/Target Deleted	--> Rows: 39242
--- HTML Removed			--> Rows: 39242
--- Rows will remain true-cased		--> Rows: 39242
--- Rows with Empty Cells Deleted	--> Rows: 39242
--- Rows Shuffled			--> Rows: 39242
--- Source Saved: UN.en-ru.train.en-filtered.en
--- Target Saved: UN.en-ru.train.ru-filtered.ru


In [108]:
!python3 MT-Preparation/filtering/filter.py UN.en-ru.val.en UN.en-ru.val.ru en ru

Dataframe shape (rows, columns): (10000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 10000
--- Duplicates Deleted			--> Rows: 9998
--- Source-Copied Rows Deleted		--> Rows: 9998
--- Too Long Source/Target Deleted	--> Rows: 4739
--- Too Short Source/Target Deleted	--> Rows: 4738
--- HTML Removed			--> Rows: 4738
--- Rows will remain true-cased		--> Rows: 4738
--- Rows with Empty Cells Deleted	--> Rows: 4738
--- Rows Shuffled			--> Rows: 4738
--- Source Saved: UN.en-ru.val.en-filtered.en
--- Target Saved: UN.en-ru.val.ru-filtered.ru


In [109]:
!python3 MT-Preparation/filtering/filter.py UN.en-ru.test.en UN.en-ru.test.ru en ru

Dataframe shape (rows, columns): (10000, 2)
--- Rows with Empty Cells Deleted	--> Rows: 10000
--- Duplicates Deleted			--> Rows: 10000
--- Source-Copied Rows Deleted		--> Rows: 10000
--- Too Long Source/Target Deleted	--> Rows: 4686
--- Too Short Source/Target Deleted	--> Rows: 4685
--- HTML Removed			--> Rows: 4685
--- Rows will remain true-cased		--> Rows: 4685
--- Rows with Empty Cells Deleted	--> Rows: 4685
--- Rows Shuffled			--> Rows: 4685
--- Source Saved: UN.en-ru.test.en-filtered.en
--- Target Saved: UN.en-ru.test.ru-filtered.ru


In [110]:
!python3 MT-Preparation/filtering/semantic_filter.py UN.en-ru.train.en-filtered.en UN.en-ru.train.ru-filtered.ru en ru 5000 #batch 0.45 #treshhold

2025-10-26 14:17:13.511377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761488233.545069   38353 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761488233.554892   38353 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761488233.578074   38353 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761488233.578105   38353 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761488233.578112   38353 computation_placer.cc:177] computation placer alr

In [91]:
!python3 MT-Preparation/filtering/semantic_filter.py UN.en-ru.val.en-filtered.en UN.en-ru.val.ru-filtered.ru en ru 5000 0.45

Traceback (most recent call last):
object address  : 0x7980ff3059c0
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr


In [92]:
!python3 MT-Preparation/filtering/semantic_filter.py UN.en-ru.test.en-filtered.en UN.en-ru.test.ru-filtered.ru en ru 5000 0.45

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/usr/local/lib/python3.12/dist-packages/torch/_subclasses/meta_utils.py", line 599, in <module>
    @dataclass(frozen=True)
     ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/dataclasses.py", line 1265, in wrap
    return _process_class(cls, init, repr, eq, order, unsafe_hash,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/dataclasses.py", line 1063, in _process_class
    _init_fn(all_init_fields,
  File "/usr/lib/python3.12/dataclasses.py", line 619, in _init_fn
    return _create_fn('__init__',
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/dataclasses.py", line 473, in _create_fn
